### GASS evolution

nb to test the evolution functions 

In [1]:
using PyCall
using DataFrames
using Printf

#rootdir = "/home/stephane/Science/ALMA/ArrayConfig/GASS"
rootdir = "/home/stephane/alma/ArrayConfig/GASS"

push!(LOAD_PATH,"$rootdir/master/src")
using GASS
using Printf

import PyPlot
import Random

coord= pyimport("astropy.coordinates")
am= pyimport("astropy.modeling.models")
mf= pyimport("astropy.modeling.fitting")

np= pyimport("numpy")


## directory
datadir = "$rootdir/master/data"
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/test"

cd(wdir)

┌ Info: Recompiling stale cache file /home/stephane/.julia/compiled/v1.1/GASS.ji for GASS [top-level]
└ @ Base loading.jl:1184


In [2]:
mutable struct _population
    age::Int32                 ## age
    subarr::Array{Array{Int,1},2}     ## {Population,Subindices}
    fitness::Array{Float64,2}  ## fitness of each subarray
    score::Array{Float64,1}    ## global score of each set
    param::Array{Dict{String,Float64},2}    ## beam and mrs for each subarray    
end

In [3]:
## fitness function for a subarray
## cfg: GASS parameters
## subarrid: subarray id (int)
## subind: subarray indices in the main Arr.
##
function _fitness_subarray(cfg, subarrid, subind)
    subarr = cfg.arr[subind,:]
    
    bl= calc_baselines(subarr)
    uv= calc_uv(bl, cfg.obs.Source_Hour_Angle ,  cfg.obs.Source_Declination)
    h , dr=  calc_dirtybeam(uv , 255, 127, robust=0.5)
    b= fit_beam(h , dr)
    mrs= calc_mrs(uv)
    
    #@printf("## subarray fitness \n")
    #@printf("## beam:")
    #println(b)
    #@printf("## MRS: %3.3f \n", mrs)
    
    res= 0
    res += cfg.wei.Weight_Spatial_Resolution[subarrid]*abs(b.ar-cfg.sub.Spatial_Resolution[subarrid])
    res += cfg.wei.Weight_Elongation[subarrid]*abs(b.e-cfg.sub.Elongation[subarrid])*sign(b.e-
        cfg.sub.Elongation[subarrid])
    res += cfg.wei.Weight_Sidelobe_Levels[subarrid]*abs(b.sidelobe-
        cfg.sub.Sidelobe_Level[subarrid])*sign(b.sidelobe-cfg.sub.Sidelobe_Level[subarrid])
    res += cfg.wei.Weight_Maximum_Recoverable_Scale[subarrid]*abs(mrs-
        cfg.sub.Maximum_Recoverable_Scale[subarrid])*sign(cfg.sub.Maximum_Recoverable_Scale[subarrid]-mrs)
    
    return(res , [b , mrs])
end

_fitness_subarray (generic function with 1 method)

In [4]:
## Creation of a population
## 

function _create_population(cfg)
    
    pop= Array{Array{Int,1},2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    fitness= Array{Float64,2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    score=  Array{Float64,1}(undef,cfg.ga.Population_Size)
    paramsub= Array{Dict{String,Float64},2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    
    subind= collect(1:cfg.obs.Antenna_Number)
    
    for i in 1:cfg.ga.Population_Size
        Random.shuffle!(subind)
        for j in 1:cfg.obs.Subarray_Number
            pop[i,j]= subind[cfg.sub.Subrange[j]]
            fitness[i,j] , res= _fitness_subarray(cfg, j, pop[i,j])
            println(i," ",j," ",fitness[i,j])
            println(res)
            paramsub[i,j]= Dict("ar"=>res[1].ar,"e"=>res[1].e, "sidelobe"=>res[1].sidelobe, "mrs"=>res[2])
        end
        score[i]= -sum(cfg.wei.Weight_Subarray[:] .* fitness[i,:])
    end
    
    _pop= _population(0, pop, fitness, score , paramsub)
    
    return(_pop)
end

_create_population (generic function with 1 method)

In [5]:
## get best parents
##

function _get_elitism(cfg, pop::_population)
    isort= reverse(sortperm(pop.score))
    
    elit= []
    for i in 1:cfg.ga.Number_Elitism
        push!(elit, [pop.subarr[isort[i],:], pop.fitness[isort[i],:], pop.score[isort[i]]])
    end
    return(elit)
end

## using tournament selection
function _get_parents(cfg, pop::_population)
    tour= cfg.ga.Tournament_Size
    
    popran= Random.shuffle!(collect(1:cfg.ga.Population_Size))
    isort= reverse(sortperm(pop.score[popran[1:tour]]))
    iparent1= popran[isort[1]]
  
    iparent2= iparent1
    while iparent1==iparent2
        popran= Random.shuffle!(collect(1:cfg.ga.Population_Size))
        isort= reverse(sortperm(pop.score[popran[1:tour]]))
        iparent2= popran[isort[1]] 
    end 
    return(iparent1 , iparent2)
end

_get_parents (generic function with 1 method)

In [6]:
function _flatten_subarray(cfg, subarr)
    subflat= zeros(Int, cfg.obs.Antenna_Number)
    
    counter= 1
    for i in 1:cfg.obs.Subarray_Number
        for j in 1:cfg.sub.Pads_Per_Subarray[i]
            subflat[counter]= subarr[i][j]
            counter += 1
        end
    end
    return(subflat)
end

## reform the subarray array..
function _wrap_subarray(cfg, sub)
    subarray= Array{Array{Int,1}}(undef,0)

    for i in 1:cfg.obs.Subarray_Number
        push!(subarray, sub[cfg.sub.Subrange[i]])
    end
    
    return(subarray)
end

## crossover using the two parents
##
function _get_crossover1(cfg, pop::_population , parents)
    nmax= cfg.obs.Antenna_Number
    p1= _flatten_subarray(cfg, pop.subarr[parents[1],:])
    p2= _flatten_subarray(cfg, pop.subarr[parents[2],:])
    child= zeros(Int, nmax)

    pivot= rand(1:nmax,2)
    pivot1= minimum(pivot) ; pivot2= maximum(pivot)
    child[pivot1:pivot2] .= p1[pivot1:pivot2]

    irange1= 1:pivot1-1 ; irange2= pivot2+1:nmax
    
    cnterp2= 1
    for i in irange1
        while (p2[cnterp2] in child)
            cnterp2 += 1
        end
        child[i]= p2[cnterp2]
    end
    for i in irange2
        while (p2[cnterp2] in child)
            cnterp2 += 1
        end
        child[i]= p2[cnterp2]
    end  
    
    # println(child)
    c= _wrap_subarray(cfg, child)
    # println(c)
    return(child)
end

_get_crossover1 (generic function with 1 method)

In [7]:
## mutation
## loop over all alleles and swap two if p is true
## 

function _get_mutation(cfg, child)
    nmax= cfg.obs.Antenna_Number
    si= copy(child)
    
    pmutation= 1-cfg.ga.Mutation_Rate
    rng = Random.MersenneTwister()
    rd= Random.rand(rng, nmax)
    # println(rd)
    for i in 1:nmax
        if rd[i] > pmutation
            iswap= Random.rand(rng, big.(1:nmax))
            value= si[iswap]
            si[iswap]= si[i]
            si[i]= value
        end
    end
    return(si)
end

_get_mutation (generic function with 1 method)

In [8]:
## Evolve one population
##

function _get_evolution(cfg, pi::_population)
    npop= cfg.ga.Population_Size
    age= pi.age + 1
    nelit= cfg.ga.Number_Elitism
    
    ## Elitism
    pelit=  _get_elitism(cfg, pi)
    
    ## parent selection, crossover and mutation
    ncross= npop-nelit
    
    crosspop= []
    for i in 1:ncross
        parent= _get_parents(cfg, pi)
        child=  _get_crossover1(cfg, pi , parent)
        mutated= _get_mutation(cfg, child)
        push!(crosspop, mutated)
        println(parent)
        println(child)
    end

    
    ## create an evolved population
    pinew= Array{Array{Int,1},2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    fitness= Array{Float64,2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    score=  Array{Float64,1}(undef,cfg.ga.Population_Size)
    paramsub= Array{Dict{String,Float64},2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    
    for i in 1:nelit
        for j in 1:cfg.obs.Subarray_Number
            pinew[i,j]= pelit[i][1][j]
            fitness[i,j], res= _fitness_subarray(cfg, j, pinew[i,j])
            ##+
            paramsub[i,j]= Dict("ar"=>res[1].ar,"e"=>res[1].e, "sidelobe"=>res[1].sidelobe, "mrs"=>res[2])
            println(i," ",j," ",fitness[i,j])
        end
        score[i]= -sum(cfg.wei.Weight_Subarray[:] .* fitness[i,:])
    end    
    
    println("fitness of crossover..")
    for i in nelit+1:npop
        println(crosspop[i-nelit])
        pwrap= _wrap_subarray(cfg, crosspop[i-nelit])
        println(pwrap)
        for j in 1:cfg.obs.Subarray_Number
            pinew[i,j]= pwrap[j]
            fitness[i,j] , res= _fitness_subarray(cfg, j, pinew[i,j])
            paramsub[i,j]= Dict("ar"=>res[1].ar,"e"=>res[1].e, "sidelobe"=>res[1].sidelobe, "mrs"=>res[2])
            println(i," ",j," ",fitness[i,j])
        end
        score[i]= -sum(cfg.wei.Weight_Subarray[:] .* fitness[i,:])
    end
    
    _pop= _population(age, pinew, fitness, score, paramsub)
    
    return(_pop)
end

_get_evolution (generic function with 1 method)

In [9]:
macro main(inpfile)
    cfg = read_cfg(inpfile , verbose=true)
    
    println("##")
    println("## Creating the population...")
    p0= _create_population(cfg)
    
    println("Elitism..")
    pelit=  _get_elitism(cfg, p0)
    
    println("Crossover...")
    parent= _get_parents(cfg, p0)
    child=  _get_crossover1(cfg, p0 , parent)
    println(child)
    
    println("Mutatis mutandis...")
    mutated= _get_mutation(cfg, child)
    
    ## evolution
    println("Evolution...")
    
    species= []
    for i in 1:cfg.ga.Number_Iterations
        #if (i % 10) == 0
            @printf("\n## Iteration: %d \n",i)
        #end
        
        p1= _get_evolution(cfg, p0)
        push!(species,p1)
        p0=p1
    end
    
    println("## Results...")
    println(species)
    
end

@main("../master/data/GA_Inputs_O-3.txt.julia")

## Input Parameters for GASS 
### Configuration file: ../master/data/O-3.cfg 
### Obs. Latitude: -23.026 
### Source Declination: -50.0 
### HA: 0.0 
### Antenna number: 50 
### Subarray number: 4 
##
## Subarray Parameters
### Pads per subarray: [30, 10, 7, 3]
### Name: ["Sub_1", " Sub_2 ", " Sub_3 ", " Sub_4"]
### AR: [2.3, 3.0, 3.5, 3.5]
### MRS: [20.0, 20.0, 10.0, 10.0]
### elongation: [1.2, 1.3, 2.0, 3.0]
### sidelobe: [10.0, 20.0, 60.0, 60.0]
##
## GA parameters
### Iterations: 5 
### Population size: 10 
### Mutation rate: 0.050 
### Tournament size: 5 
### Elitism: 5 
##
## Weights
### Subarray weights: [0.25, 0.25, 0.25, 0.25]
### AR weights: [0.25, 0.25, 0.25, 0.25]
### MRS weights: [0.25, 0.25, 0.25, 0.25]
### elongation weights: [0.25, 0.25, 0.25, 0.25]
### sidelobe weights: [0.25, 0.25, 0.25, 0.25]
##
##
## Creating the population...
1 1 2.4068290098497114
Any[synthbeam(2.75787, 2.23273, 2.48145, 1.2352, 7.01664), 7.60597]
1 2 0.667308034796295
Any[synthbeam(2.55538, 3.378

9 3 -10.534340968879315
9 4 16.538230907693958
[48, 41, 27, 43, 12, 5, 30, 42, 26, 18, 49, 19, 23, 8, 20, 9, 50, 36, 33, 2, 14, 28, 40, 45, 17, 25, 32, 6, 37, 10, 38, 21, 7, 24, 1, 35, 22, 31, 46, 34, 15, 47, 44, 11, 4, 29, 3, 13, 39, 16]
Array{Int64,1}[[48, 41, 27, 43, 12, 5, 30, 42, 26, 18, 49, 19, 23, 8, 20, 9, 50, 36, 33, 2, 14, 28, 40, 45, 17, 25, 32, 6, 37, 10], [38, 21, 7, 24, 1, 35, 22, 31, 46, 34], [15, 47, 44, 11, 4, 29, 3], [13, 39, 16]]
10 1 2.1770834604179945
10 2 0.584801397628866
10 3 -11.766764131206038
10 4 8.211477854520973

## Iteration: 2 
(7, 1)
[44, 11, 37, 22, 10, 30, 42, 26, 18, 49, 2, 43, 8, 20, 50, 36, 33, 19, 14, 28, 40, 45, 17, 3, 32, 48, 16, 23, 25, 5, 7, 29, 4, 38, 12, 41, 46, 39, 1, 31, 6, 27, 47, 35, 9, 13, 24, 34, 15, 21]
(7, 8)
[11, 37, 47, 42, 26, 2, 13, 20, 9, 39, 27, 4, 22, 12, 46, 45, 16, 49, 18, 29, 44, 33, 30, 10, 38, 25, 14, 50, 36, 43, 19, 28, 40, 17, 35, 3, 32, 48, 23, 5, 7, 41, 1, 31, 6, 8, 24, 34, 21, 15]
(7, 8)
[11, 37, 47, 10, 30, 42, 26, 

10 1 1.744877183749404
10 2 0.4732621416272358
10 3 -21.043978572966733
10 4 -10.926851994471377

## Iteration: 4 
(1, 10)
[37, 2, 42, 21, 26, 22, 49, 18, 44, 33, 30, 10, 24, 20, 50, 36, 43, 19, 14, 28, 40, 45, 8, 3, 32, 48, 16, 23, 25, 5, 7, 29, 4, 38, 12, 46, 39, 1, 17, 6, 27, 13, 47, 35, 9, 41, 11, 31, 34, 15]
(10, 1)
[37, 2, 42, 50, 21, 26, 36, 27, 22, 49, 18, 44, 33, 30, 10, 24, 20, 43, 6, 19, 14, 28, 40, 45, 8, 35, 3, 32, 48, 16, 23, 25, 5, 7, 29, 4, 38, 12, 41, 46, 39, 1, 17, 13, 47, 9, 11, 31, 34, 15]
(10, 1)
[37, 2, 42, 50, 21, 26, 22, 49, 18, 44, 33, 30, 10, 24, 20, 9, 11, 36, 43, 19, 14, 28, 40, 45, 8, 35, 3, 32, 48, 16, 23, 25, 5, 7, 29, 4, 38, 12, 41, 46, 39, 1, 17, 6, 27, 13, 47, 31, 34, 15]
(3, 1)
[37, 42, 21, 26, 39, 27, 22, 46, 49, 18, 44, 2, 33, 13, 20, 9, 50, 36, 43, 19, 14, 28, 40, 45, 17, 35, 3, 32, 48, 16, 23, 25, 5, 7, 29, 4, 38, 12, 41, 30, 10, 24, 8, 1, 6, 47, 11, 31, 34, 15]
(1, 10)
[37, 2, 42, 21, 26, 22, 31, 50, 36, 19, 7, 4, 12, 46, 39, 45, 16, 49, 18, 29, 

Any[_population(1, Array{Int64,1}[[44, 11, 37, 22, 47, 10, 30, 42, 26, 18, 49, 2, 43, 8, 20, 9, 50, 36, 33, 19, 14, 28, 40, 45, 17, 35, 3, 32, 48, 16] [23, 25, 5, 7, 29, 4, 38, 12, 41, 46] [39, 1, 31, 6, 27, 13, 24] [34, 15, 21]; [37, 2, 42, 18, 21, 26, 19, 7, 36, 39, 27, 4, 22, 12, 46, 44, 16, 49, 10, 6, 29, 45, 25, 14, 24, 5, 28, 8, 30, 1] [20, 40, 48, 43, 50, 3, 23, 13, 17, 32] [47, 35, 33, 9, 41, 38, 11] [31, 34, 15]; [48, 41, 42, 27, 9, 23, 40, 12, 5, 2, 18, 25, 19, 32, 17, 6, 8, 37, 28, 36, 50, 33, 30, 10, 38, 45, 21, 7, 43, 24] [1, 35, 22, 31, 46, 34, 15, 47, 44, 11] [4, 29, 3, 26, 13, 20, 49] [14, 39, 16]; [27, 12, 30, 40, 4, 11, 28, 43, 32, 31, 46, 10, 42, 50, 5, 25, 26, 8, 3, 33, 38, 6, 49, 45, 18, 13, 9, 34, 47, 24] [23, 7, 39, 2, 44, 36, 1, 19, 21, 29] [17, 41, 37, 16, 14, 48, 35] [20, 15, 22]; [8, 30, 45, 28, 33, 29, 20, 34, 32, 18, 39, 31, 2, 19, 46, 38, 4, 47, 23, 36, 48, 42, 14, 43, 6, 44, 11, 37, 50, 24] [13, 41, 25, 16, 15, 22, 12, 49, 3, 10] [40, 1, 5, 7, 17, 26, 35]

, 5.42148, 5.77993, 5.72812, 6.77704, 7.43817], Dict{String,Float64}[Dict("mrs"=>8.99568,"ar"=>2.55378,"e"=>1.04629,"sidelobe"=>6.2001) Dict("mrs"=>12.1474,"ar"=>2.77185,"e"=>1.35607,"sidelobe"=>12.9351) Dict("mrs"=>9.6776,"ar"=>3.26181,"e"=>1.25604,"sidelobe"=>10.1593) Dict("mrs"=>27.4404,"ar"=>8.79254,"e"=>1.5869,"sidelobe"=>4.80338); Dict("mrs"=>5.96068,"ar"=>2.33709,"e"=>1.00905,"sidelobe"=>6.42021) Dict("mrs"=>8.87755,"ar"=>3.09968,"e"=>1.36511,"sidelobe"=>10.6058) Dict("mrs"=>24.855,"ar"=>3.53748,"e"=>1.24949,"sidelobe"=>7.56104) Dict("mrs"=>11.934,"ar"=>5.2176,"e"=>1.0,"sidelobe"=>10.791); Dict("mrs"=>5.96068,"ar"=>2.34332,"e"=>1.02971,"sidelobe"=>6.45933) Dict("mrs"=>8.87755,"ar"=>3.09968,"e"=>1.36511,"sidelobe"=>10.6058) Dict("mrs"=>24.855,"ar"=>3.64565,"e"=>1.07125,"sidelobe"=>8.83126) Dict("mrs"=>11.934,"ar"=>5.2176,"e"=>1.0,"sidelobe"=>10.791); Dict("mrs"=>8.99568,"ar"=>2.72249,"e"=>1.01617,"sidelobe"=>7.34377) Dict("mrs"=>7.43818,"ar"=>2.34331,"e"=>1.12698,"sidelobe"=>10.4

sidelobe"=>16.9681); Dict("mrs"=>35.9339,"ar"=>2.36142,"e"=>1.08077,"sidelobe"=>6.21895) Dict("mrs"=>8.87755,"ar"=>3.04553,"e"=>1.57399,"sidelobe"=>10.1431) Dict("mrs"=>55.3858,"ar"=>4.87767,"e"=>2.36771,"sidelobe"=>9.14632) Dict("mrs"=>27.4404,"ar"=>8.79254,"e"=>1.5869,"sidelobe"=>4.80338); Dict("mrs"=>5.62006,"ar"=>2.39209,"e"=>1.04469,"sidelobe"=>5.60574) Dict("mrs"=>43.9754,"ar"=>3.22287,"e"=>1.08588,"sidelobe"=>11.7955) Dict("mrs"=>7.81937,"ar"=>2.85283,"e"=>1.00768,"sidelobe"=>11.7239) Dict("mrs"=>14.7838,"ar"=>8.31039,"e"=>1.37211,"sidelobe"=>90.3574); Dict("mrs"=>3.46414,"ar"=>2.32922,"e"=>1.00568,"sidelobe"=>6.74588) Dict("mrs"=>11.3439,"ar"=>3.24288,"e"=>1.1016,"sidelobe"=>12.2226) Dict("mrs"=>7.76301,"ar"=>3.38745,"e"=>1.02047,"sidelobe"=>15.698) Dict("mrs"=>27.4404,"ar"=>8.79254,"e"=>1.5869,"sidelobe"=>4.80338); Dict("mrs"=>8.54469,"ar"=>2.36671,"e"=>1.07445,"sidelobe"=>6.03404) Dict("mrs"=>8.87755,"ar"=>3.09968,"e"=>1.36511,"sidelobe"=>10.6058) Dict("mrs"=>48.7204,"ar"=>3.